## Search the Hengduan Database 
We hope to find many species for which collections are available from multiple disjunct regions sampled between 1997-2004. 

In [4]:
#conda install pandas
#conda install beautifulsoup4
#pip install folium

In [11]:
from concurrent.futures import ThreadPoolExecutor
import requests
import pandas as pd
from bs4 import BeautifulSoup
import folium
import time

### The Hengduan API structure

In [12]:
# specimen search page
# http://hengduan.huh.harvard.edu/fieldnotes/specimens/search/search.zpt?
#    dna_collection=on&
#    st=pedicularis&
#    action=search&
#    submit_button=Search

In [13]:
# individual specimen page
# http://hengduan.huh.harvard.edu/fieldnotes/specimens/search/specimen_detail.zpt?
#    specimen_id=718

### A class object to search and parse Hengduan data

In [14]:
class Hengduan:
    "Search the Hengduan database and return a dataframe"
    
    def __init__(self, taxon, dna=True,):
        # class globals
        baseurl = "http://hengduan.huh.harvard.edu/fieldnotes/"
        self.search_url = baseurl + "specimens/search/search.zpt"
        self.specimen_url = baseurl + "specimens/search/specimen_detail.zpt"
        
        # class attrs
        self.taxon = taxon
        self.dna = ["on" if True else "off"][0]
        self.data = None
        
        # do search and fill database
        if self._get_search_data():
            self._fill_data_coords()
        
        
    def _search_request(self):
        # slow it down by waiting one second
        time.sleep(1)
        res = requests.get(
            url=self.search_url,
            params={
                "dna_collection": self.dna, 
                "st": self.taxon,
                "action": "search", 
                "submit_button": "Search",
            }
        )
        res.raise_for_status()
        return BeautifulSoup(res.text, "html5lib")
       
        
    def _get_search_data(self):
        soup = self._search_request()
        table = soup.find('table', attrs={"class": "listing", "id": "angio_table"})
        if table:
            headers = [header.text for header in table.find_all('th')]
        # if no record then return 0
        else:
            return 0
        
        headers.extend(["specimen-id"])
        rows = []
        for row in table.find_all('tr')[1:]:
            tds = row.find_all('td')
            row = [val.text.strip() for val in tds]
            tmp = [i.a for i in tds][2]
            spid = (tmp.attrs['href'].split("=")[-1])
            row.extend([spid])
            rows.append(row)
        self.data = pd.DataFrame(
            rows, 
            columns=["family", "taxon", "cid", "cdate", "", "sid"]
        )
        self.data['year'] = (
            self.data["cdate"]
             .apply(str.split)
             .apply(lambda x: x[-1])
            )
        self.data = self.data.drop(["cdate", ""], axis=1)
        # add shortname ref
        self.data["shortname"] = (
            self.data
            .taxon
            .apply(str.split)
            .apply("-".join)
        )
        # on success return 1
        return 1
        
    def _specimen_request(self, spid):
        res = requests.get(
            url=self.specimen_url,
            params={
                "specimen_id": spid,
            }
        )
        res.raise_for_status()
        return BeautifulSoup(res.text, "html5lib")

        
    def _get_coordinates(self, specid):
        soup = self._specimen_request(specid)
        textlocs = soup.find(id="locality")
        text = textlocs.find_all("td")[1].text.split("\n")
        descr = " ".join([i.strip() for i in text][1:4])
        tmp0, tmp1 = text[-3].strip().split("°")
        tmp0, tmp1
        tmp1, tmp2 = tmp1.split("\'")
        tmp2 = tmp2.lstrip(";").rstrip(";").replace('"', '')
        point = "-".join([tmp0, tmp1, tmp2])
        eastwest = self._convert_gps(point)

        tmp0, tmp1 = text[-4].strip().split("°")
        tmp1, tmp2 = tmp1.split("\'")
        tmp2 = tmp2.lstrip(";").rstrip(",").replace('"', '')
        point = "-".join([tmp0, tmp1, tmp2])
        northsouth = self._convert_gps(point)
        return (northsouth, eastwest)

        
    @staticmethod
    def _convert_gps(tude):
        multiplier = 1 if tude[-1] in ['N', 'E'] else -1
        return multiplier * sum(float(x) / 60 ** n for n, x in enumerate(tude[:-1].split('-')))  
    
    
    def _fill_data_coords(self):
        with ThreadPoolExecutor(max_workers=4) as executor:
            jobs = [executor.submit(
                self._get_coordinates, specid) for specid in self.data.sid]
            res = [i.result() for i in jobs]
        
        self.data['latitude'] = [i[0] for i in res]
        self.data['longitude'] = [i[1] for i in res]
        
        
    def count_by_maxyear(self, year):
        try:
            return (
             self.data[self.data.year.astype(int) <= year]
             .sort_values(by=["year", "shortname"])
             .groupby('shortname')
             .apply(len)
             .sort_values(ascending=False)
            )
        except TypeError:
            return None
        
            
    def filter_by_max_year(self, year):
        try:
            return (
             self.data[self.data.year.astype(int) <= year]
             .sort_values(by=["year", "shortname"])
            )
        except TypeError:
            return None


### Example usage

In [15]:
# get dataframe
coll = Hengduan(taxon="Ped cranolopha", dna=True)

In [16]:
# see first N records
coll.filter_by_max_year(1998)

,family,taxon,cid,sid,year,shortname,latitude,longitude
0,Orobanchaceae,Pedicularis\n cranolopha,27709,595,1997,Pedicularis-cranolopha,30.850278,101.276667
1,Orobanchaceae,Pedicularis\n cranolopha,27821,707,1997,Pedicularis-cranolopha,31.660000,100.712778
2,Orobanchaceae,Pedicularis\n cranolopha,28522,1413,1998,Pedicularis-cranolopha,29.141111,99.928333
3,Orobanchaceae,Pedicularis\n cranolopha,28746,1640,1998,Pedicularis-cranolopha,29.103056,99.631944


# MAP

### A class object to map lat long data

In [17]:
# make size a function of year
y = 2018
x = 1997
import numpy as np

def yearsize(x):
    diff = x - 1995
    ndiff = 1 - (diff / 25)
    return 12 * np.sqrt(ndiff)
    
    
yearsize(x), yearsize(y)

(11.509995655950528, 3.3941125496954276)

In [18]:
class Map:
    def __init__(self, hlocs=None, nlocs=None, height=None, width=None):
                 #color=None):
        
        # store attributes
        self.nlocs = nlocs
        self.hlocs = hlocs
        self.alldata = pd.concat([self.nlocs, self.hlocs], sort=False)
        #self.color = color
        
        # create map with zoom x and location y
        if not width:
            width = 450
        if not height:
            height = width
        
        self.map = folium.Map(
            location=[30.3849, 101.0083],
            zoom_start=6.5,
            #location=[self.alldata.latitude.mean(), self.alldata.longitude.mean()],
            tiles='Stamen Terrain'
        )

        # add points
        if isinstance(nlocs, pd.DataFrame):
            self.add_nlocs()
        if isinstance(hlocs, pd.DataFrame):
            self.add_hlocs()
            
    def draw(self):
        # fit bounds of map to show all points
        latlongtuples = [
            self.alldata[["latitude", "longitude"]].iloc[i].tolist() for i in self.alldata.index
        ]
        #self.map.fit_bounds(latlongtuples)
        return self.map
       
    def add_nlocs(self):
        for idx in self.nlocs.index:
            lat, long = self.nlocs.iloc[idx].latitude, self.nlocs.iloc[idx].longitude
            folium.CircleMarker(
                location=[lat, long],
                color='rgba(85.1%,37.3%,0.8%,1.000)',
                #color= self.color,
                fill=True,
                fill_color='rgba(85.1%,37.3%,0.8%,1.000)',
                #fill_color=self.color,
                radius= 5, #2 * (1 - (self.nlocs.iloc[idx].year.astype(int) / 2018)),
                popup=" | ".join(self.nlocs.iloc[idx][["shortname", "cid", "year"]].tolist()),
                #icon=folium.Icon(color="blue", icon="circle", prefix="fa"),
            ).add_to(self.map)
 
    def add_hlocs(self):
        for idx in self.hlocs.index:
            lat, long = self.hlocs.iloc[idx].latitude, self.hlocs.iloc[idx].longitude
            folium.CircleMarker(
                location=[lat, long],
                color='rgba(45.9%,43.9%,70.2%,1.000)',
                #color= self.color,
                fill=True,
                fill_color='rgba(45.9%,43.9%,70.2%,1.000)',
                #fill_color= self.color,
                radius=yearsize(int(self.hlocs.iloc[idx].year)),
                popup=" | ".join(self.hlocs.iloc[idx][["shortname", "cid", "year"]].tolist()),
                #icon=folium.Icon(color="red", icon="circle", prefix="fa",
            ).add_to(self.map)
 

In [19]:
import toyplot

a = toyplot.color.brewer.palette("Dark2")
toyplot.color.to_css(a[2])

'rgba(45.9%,43.9%,70.2%,1.000)'

In [21]:
# draw historical (red) and newer collections (blue) (I think the colors are
#wrong here, so figure this out later...)
m = Map(hlocs=coll.filter_by_max_year(1998), nlocs=coll.data).draw()
m.add_child(folium.LatLngPopup())
#m.save("")

In [98]:
co = Hengduan("ped rhinanth", True)
# draw historical (red) and newer collections (blue)
m = Map(hlocs=co.filter_by_max_year(1997), nlocs=co.data)
m.draw()
m.map.save(outfile="../example.html")

### Load our 2018-19 collections into a DataFrame

In [22]:
class New:
    def __init__(self, df):
        self._load_2018_dataframe(df)
        
    def subset(self, taxon):
        return self.data[self.data.shortname.apply(lambda x: taxon in x)].reset_index()

    def _load_2018_dataframe(self, df):
        # load Data
        data = pd.read_csv(df)

        # drop unidentified
        data = data[data.species_epithet.notna()]

        # select just the columns we want
        data = data[["accession", "locality", "date", "latitude", "longitude", "genus", "species_epithet"]]
        data["shortname"] = data[["genus", "species_epithet"]].apply(lambda x: '-'.join(x), axis=1)

        # convert lat longs to decimals
        def convert_gps(tude):
            deg, _ = tude.split("°")
            minu, _ = _.split("'")
            seco = _.split('"')[0]
            tude = "-".join([deg, minu, seco])
            return sum(float(x) / 60 ** n for n, x in enumerate(tude.split('-')))  

        data["lat"] = data.latitude.apply(convert_gps)
        data["long"] = data.longitude.apply(convert_gps)
        data.head()

        data["year"] = data.date.apply(lambda x: x.split("/")[-1])
        data["cid"] = data["accession"]

        # convert label names to match with Hengduan object
        data = data.drop(columns=["latitude", "longitude", "genus", "species_epithet", "date", "accession"])
        data = data.rename({"lat": "latitude", "long": "longitude", "locality": "sid"}, axis='columns')
        self.data = data

In [23]:
# create an instance to hold all the 2018-19 data
newdata = New("/Users/jaredmeek/Desktop/hengduan/fieldnotes/Fieldnotes_Master.csv")

In [9]:
# trace csv reading error if it ever happens...
#df = pd.read_csv(
    #"/Users/jaredmeek/Desktop/hengduan/fieldnotes/Fieldnotes_Master.csv")
#def func(tude):
    #deg, _ = tude.split("°")
    #minu, _ = _.split("'")
    #seco = _.split('"')[0]
#for idx in range(999):
    #try:
        #func(df.latitude[idx])
    #except ValueError:
        #print(df.iloc[idx])
        #print(idx)

In [24]:
# get a subset data set for a specific taxon
newdata.subset("longiflora").head()

,index,sid,shortname,latitude,longitude,year,cid
0,3,2.0,Pedicularis-longiflora,27.959194,99.707167,2018,DE4
1,53,14.0,Pedicularis-longiflora,29.144722,100.034778,2018,DE54
2,60,16.0,Pedicularis-longiflora,29.334861,100.099417,2018,DE61
3,131,31.0,Pedicularis-longiflora,30.419167,101.548056,2018,DE132
4,140,33.0,Pedicularis-longiflora,30.577028,101.417972,2018,DE141


## Phylogeography

In [9]:
crano = Map(
    #width="50%", 
    #height="40%",
    nlocs=newdata.subset("cranolopha"),
    hlocs=Hengduan("Ped cranolopha", True),
).draw()

crano.save("/Users/jaredmeek/Desktop/hengduan/maps/pedmaps_python/cranolopha.html")

TypeError: cannot concatenate object of type "<class '__main__.Hengduan'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [ ]:
integ = Map(
    #width="50%", 
    #height="40%",
    nlocs=newdata.subset("integrifolia"),
    hlocs=Hengduan("Ped integrifolia", True),
).draw()

integ.save("/Users/jaredmeek/Desktop/hengduan/maps/pedmaps_python/integrifolia.html")

In [ ]:
kansu = Map(
    #width="50%", 
    #height="40%",
    nlocs=newdata.subset("kansuensis"),
    hlocs=Hengduan("Ped kansuensis", True),
).draw()

kansu.save("/Users/jaredmeek/Desktop/hengduan/maps/pedmaps_python/kansuensis.html")

In [ ]:
lachno = Map(
    #width="50%", 
    #height="40%",
    nlocs=newdata.subset("lachnoglossa"),
    hlocs=Hengduan("Ped lachnoglossa", True),
).draw()

lachno.save("/Users/jaredmeek/Desktop/hengduan/maps/pedmaps_python/lachnoglossa.html")

In [ ]:
longi = Map(
    #width="50%", 
    #height="40%",
    nlocs=newdata.subset("longiflora"),
    hlocs=Hengduan("Ped longiflora", True),
).draw()

longi.save("/Users/jaredmeek/Desktop/hengduan/maps/pedmaps_python/longiflora.html")

In [ ]:
rhinanth = Map(
    #width="50%", 
    #height="40%",
    nlocs=newdata.subset("rhinanthoides"),
    hlocs=Hengduan("Ped rhinanthoides", True),
).draw()

rhinanth.save("/Users/jaredmeek/Desktop/hengduan/maps/pedmaps_python/rhinanthoides.html")

In [29]:
# how to concatenate?
#phylogeo = pd.concat([crano, integ, kansu, lachno, longi, rhinanth])

## Maps for historical-contemporary species

In [42]:
# example map old and new 
Map(
    #width="50%", 
    #height="40%",
    nlocs=newdata.subset("longiflora"),
    hlocs=Hengduan("Ped longiflora", True).filter_by_max_year(1998),
).draw()

In [43]:
# this returns a Hengduan object  
peds = Hengduan("Pedicularis ", dna=True)

# this attribute of the object has the unfiltered data
peds.data

# this function call from the object return a DF
old = peds.filter_by_max_year(1998)

In [49]:
# visualize old DataFrame
print(old.to_string())

            family                                              taxon    cid   sid  year                                  shortname   latitude   longitude
0    Orobanchaceae                                        Pedicularis  27501   384  1997                                Pedicularis  30.040278  101.838333
1    Orobanchaceae                                        Pedicularis  27567   452  1997                                Pedicularis  30.015278  101.859444
35   Orobanchaceae                  Pedicularis\n                anas  27904   790  1997                           Pedicularis-anas  31.706111  102.313889
36   Orobanchaceae                  Pedicularis\n                anas  27966   852  1997                           Pedicularis-anas  31.850278  102.670278
90   Orobanchaceae          Pedicularis\n                anthemifolia  27837   723  1997                   Pedicularis-anthemifolia  31.725000  100.744444
151  Orobanchaceae       Pedicularis\n                cheilanthifolia 

In [50]:
#Create dictionary for all species with collections older than 1998
listofnames = [ 
    'anas',
    'axillaris',
    'cheilanthifolia',
    'confertiflora',
    'cranolopha',
    'cristatella',
    'cyathophylla',
    'davidii',
    'decorissima',
    'densispica',
    'dichotoma',
    'dolichocymba',
    'dolichoglossa',
    'elwesii',
    'ingens',
    'integrifolia',
    'kansuensis',
    'lachnoglossa',
    'lasiophrys',
    'likiangensis',
    'lineata',
    'longiflora',
    'lyrata',
    'metaszetschuanica',
    'petitmenginii',
    'przewalskii',
    'pseudomelampyriflora',
    'rex',
    'rhinanthoides',
    'rhodotricha',
    'rhynchodonta',
    'rudis',
    'rupicola',
    'semitorta',
    'siphonantha',
    'souliei',
    'superba',
    'tatsienensis',
    'tenera',
    'thamnophila'
]

len(listofnames)

40

In [51]:
import os

home = os.path.expanduser("~")
proj = "Desktop/hengduan/maps/pedmaps_python/h-cmaps/"

path = os.path.join(home, proj)
path

'/Users/jaredmeek/Desktop/hengduan/maps/pedmaps_python/h-cmaps/'

In [52]:
# combine Hengduan and New dataframes

def insert_epithet(name, newdata, records):
    """
    name is a string.
    newdata is a object loaded from CSV.
    records is a DataFrame.
    """
    comb = pd.concat([newdata.subset(name), records[name]], sort=True)
    comb = comb.drop(columns=["family", "taxon", "index"])
    comb.to_csv(path + "{}.csv".format(name))


In [53]:
records = {}
for taxon in listofnames:
    # store H object
    print(taxon)
    
    # scrape all data for this taxon to create a Heng object
    scraped = Hengduan("Pedicularis " + taxon)
    
    # limit scraped data to old points and return as a DF
    tmpold = scraped.filter_by_max_year(1998)
    
    # store DF in the records diction
    records[taxon] = tmpold
    
    # save csv to file that combines data from CSV obj and DF
    # ing our data with hengduan
    insert_epithet(taxon, newdata, records)
    
    # draw and save map
    imap= Map(hlocs=records[taxon], nlocs=newdata.subset(taxon))
    imap.map.save("/Users/jaredmeek/Desktop/hengduan/maps/pedmaps_python/h-cmaps/" + taxon + ".html")

anas
axillaris
cheilanthifolia
confertiflora
cranolopha
cristatella
cyathophylla
davidii
decorissima
densispica
dichotoma
dolichocymba
dolichoglossa
elwesii
ingens
integrifolia
kansuensis
lachnoglossa
lasiophrys
likiangensis
lineata
longiflora
lyrata
metaszetschuanica
petitmenginii
przewalskii
pseudomelampyriflora
rex
rhinanthoides
rhodotricha
rhynchodonta
rudis
rupicola
semitorta
siphonantha
souliei
superba
tatsienensis
tenera
thamnophila


## Extras to figure out later

In [ ]:
#broken
co = Hengduan("ped rhinanth", True)
# draw historical (red) and newer collections (blue)
m = Map(hlocs=co.filter_by_max_year(1998), nlocs=co.data)
m.draw()
m.map.save(outfile="./reduced.html")
m.save("/Users/jaredmeek/Desktop/")

In [22]:
# combine Hengduan 'records' dataframes

newdf = pd.concat([i.data[i.data.year.astype(int) < 1998] for i in records.values()])
newdf.to_csv("path")

In [42]:
records['cranolopha'].data.head()

KeyError: 'cranolopha'

In [43]:
# get a subset data set for a specific taxon
newdata.subset("siphonantha").head()

,index,sid,shortname,latitude,longitude,year,cid
0,5,2.0,Pedicularis-siphonantha,27.959194,99.707167,2018,DE6
1,19,6.0,Pedicularis-siphonantha,28.585861,99.837222,2018,DE20
2,44,12.0,Pedicularis-siphonantha,29.123889,99.996667,2018,DE45
3,48,14.0,Pedicularis-siphonantha,29.144722,100.034778,2018,DE49
4,74,18.0,Pedicularis-siphonantha,30.010972,100.316167,2018,DE75


In [19]:
obj = records['siphonantha']
obj.filter_by_max_year(1998)

KeyError: 'siphonantha'

In [17]:
Map(
    nlocs=newdata.subset("siphonantha"), 
    hlocs=records["siphonantha"].filter_by_max_year(1998),
).draw()

KeyError: 'siphonantha'

In [ ]:
# figure this out later, if it's even possible
#all_ped = Map(
    #width="50%", 
    #height="40%",
    #nlocs=newdata.subset("Pedicularis "),
    #hlocs=Hengduan("Pedicularis ", True).filter_by_max_year(1998),
#).draw()

#m.save("/Users/jaredmeek/Desktop/hengduan/maps/pedmaps_python/all_ped.html")